In [61]:
import os
import azureml
from azureml.core import Experiment
from azureml.core import Workspace, Run
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Dataset,Model

subscription_id = key
resource_group = 'ml'
workspace_name = 'project'

workspace = Workspace(subscription_id, resource_group, workspace_name)

'''
dataset = Dataset.get_by_name(workspace, name='tweets')
dataset=dataset.to_pandas_dataframe()
'''

"\ndataset = Dataset.get_by_name(workspace, name='tweets')\ndataset=dataset.to_pandas_dataframe()\n"

In [68]:
from azureml.core.resource_configuration import ResourceConfiguration


model = Model.register(workspace=workspace,
                       model_name='tweetsMulti',                # Name of the registered model in your workspace.
                       model_path='./words',  # Local file to upload and register as a model.
                       model_framework=Model.Framework.CUSTOM,  # Framework used to create the model.
                       model_framework_version='2',             # Version of scikit-learn used to create the model.
                       
                       resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5)
                       )

print('Name:', model.name)
print('Version:', model.version)

Registering model tweetsMulti
Name: tweetsMulti
Version: 2


In [69]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create the environment
myenv = Environment(name="myenv")
conda_dep = CondaDependencies()

# Define the packages needed by the model and scripts
conda_dep.add_conda_package("tensorflow")
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("scikit-learn")
# You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("keras")
conda_dep.add_pip_package("gensim")
conda_dep.add_pip_package("nltk")

# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep


In [70]:
with open('score.py') as f:
    print(f.read())

#!/usr/bin/env python
# coding: utf-8

# In[58]:


from keras.models import load_model
from keras.layers import LSTM, Dense, Embedding, Dropout, Activation,Conv1D,MaxPooling1D
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
from nltk.stem import PorterStemmer 
import string
import re
import json
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from multiprocessing import cpu_count
stop_words = set(stopwords.words('english'))
import numpy as np
import pandas as pd
import azureml.core
#from gensim.models import Word2Vec
import pickle
from keras.preprocessing.sequence import pad_sequences
from azureml.core import Workspace , Dataset, Model





def init():
    
    
   
    global modelKeras
    #global modelGensim
    global tokenizer
    # Replace filename if needed.
    model_path = Model.get_model_path(model_name='multiple',_workspace=workspace)
    # Deserialize the model file back into a sklearn model.
 

In [71]:
from azureml.core import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException


service_name = 'my-custom-env-service'

# Remove any existing service under the same name.
try:
    Webservice(workspace, service_name).delete()
except WebserviceException:
    pass

inference_config = InferenceConfig(entry_script='score.py', environment=myenv)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=workspace,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

Running...........................................................................................................................................................................................................................................
TimedOut


ERROR - Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: a098a93c-bdba-4c2e-b3dc-069d0f87bd51
More information can be found using '.get_logs()'
Error:
{
  "code": "DeploymentTimedOut",
  "statusCode": 504,
  "message": "The deployment operation polling has TimedOut. The service creation is taking longer than our normal time. We are still trying to achieve the desired state for the web service. Please check the webservice state for the current webservice health. You can run print(service.state) from the python SDK to retrieve the current state of the webservice."
}

ERROR - Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: a098a93c-bdba-4c2e-b3dc-069d0f87bd51
More information can be found using '.get_logs()'
Error:
{
  "code": "DeploymentTimedOut",
  "statusCode": 504,
  "message": "The deployment operation polling has TimedOut. The service creation is taking lo

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: a098a93c-bdba-4c2e-b3dc-069d0f87bd51
More information can be found using '.get_logs()'
Error:
{
  "code": "DeploymentTimedOut",
  "statusCode": 504,
  "message": "The deployment operation polling has TimedOut. The service creation is taking longer than our normal time. We are still trying to achieve the desired state for the web service. Please check the webservice state for the current webservice health. You can run print(service.state) from the python SDK to retrieve the current state of the webservice."
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Unhealthy\nOperation ID: a098a93c-bdba-4c2e-b3dc-069d0f87bd51\nMore information can be found using '.get_logs()'\nError:\n{\n  \"code\": \"DeploymentTimedOut\

In [73]:

print(service.state)
print("scoring URI: " + service.scoring_uri)

Unhealthy
scoring URI: http://9aef3626-5af9-4e0a-b503-b1fc9d78387a.eastus.azurecontainer.io/score
